In [2]:
from os import listdir,mkdir
from os.path import isfile, join, splitext,exists
from scipy.io import wavfile
from scipy import signal
import sys
import librosa
import matplotlib.pyplot as plt
import numpy as np
from madmom.audio.filters import MelFilterbank 
from pydub import AudioSegment
import os
import soundfile as sf


path_giant_audio="Dataset/Key/GS/Audio"
path_giant_ann="Dataset/Key/GS/Ann"
path_giant_image="Dataset/Key/GS/Image_spec"
path_gtzan_audio="Dataset/Key/GTZAN/Audio"
path_gtzan_ann="Dataset/Key/GTZAN/Ann"
path_gtzan_image="Dataset/Key/GTZAN/Image_spec"
lst_name_chords=['A major','Bb major','B major','C major','Db major','D major','Eb major','E major','F major','Gb major','G major','Ab major','A minor','Bb minor','B minor','C minor','Db minor','D minor','Eb minor','E minor','F minor','Gb minor','G minor','Ab minor']
lst_chords=range(len(lst_name_chords))

#run this line in order to download the giantsteps key dataset

!git clone https://github.com/GiantSteps/giantsteps-key-dataset.git

In [2]:
path=[path_giant_ann,path_gtzan_ann,path_gtzan_audio,path_giant_audio]
def create_folder(path):
    print("Creating the folder for key dataset...")
    for p in path:
        list_path=p.split('/')
        folder=list_path[0]
        if not exists(folder):
            os.mkdir(folder)
        for n in range(len(list_path)-1):
            folder=folder+f"/{list_path[n+1]}"
            if not exists(folder):
                os.mkdir(folder)   
                
                

create_folder(path)

Dataset
Dataset/Key
Dataset/Key/GS
Dataset
Dataset/Key
Dataset/Key/GTZAN
Dataset
Dataset/Key
Dataset/Key/GTZAN
Dataset
Dataset/Key
Dataset/Key/GS


In [3]:
#please run the file audio "audio_dl.sh" in "giantsetps-key-dataset" folder before running this part

def move_data_gs(path,dst_audio,dst_ann):
    
    path_ann=path+"/annotations/key"
    onlyfiles = [f for f in listdir(path_ann) if isfile(join(path_ann, f))]
    for f in onlyfiles:
        os.rename(path_ann+f"/{f}",dst_ann+f"/{f}")
    path_audio=path+"/audio"
    new_path_audio=dst_audio
    onlyfiles = [f for f in listdir(path_audio) if isfile(join(path_audio, f))]
    for f in onlyfiles:
        os.rename(path_audio+f"/{f}",dst_audio+f"/{f}")
        

move_data_gs('giantsteps-key-dataset',path_giant_audio,path_giant_ann)

In [13]:
#regarding the GTZAN project: download the dataset from the link "http://opihi.cs.uvic.ca/sound/genres.tar.gz"
#please extract the file in the folder "GTZAN"
#After that delete the file txt "Delete This.txt"
#!git clone "https://github.com/alexanderlerch/gtzan_key.git"
def move_data_gtzan(path,dst_audio,dst_ann):
    onlydir = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    for d in onlydir:
        onlydir2=[f for f in listdir(path+"/"+d) if os.path.isdir(join(path+"/"+d, f))]
        for f in onlydir2:
            onlyfiles=[t for t in listdir(path+"/"+d+"/"+f) if isfile(join(path+"/"+d+"/"+f, t)) if t.endswith('.wav')]
            for t in onlyfiles:
                os.rename(path+f"/{d}/{f}/{t}",dst_audio+f"/{t}")
                
    '''onlydir = [name for name in os.listdir("gtzan_key/gtzan_key/genres") if os.path.isdir(os.path.join("gtzan_key/gtzan_key/genres", name))]
    for d in onlydir:
        onlyfiles=[f for f in listdir("gtzan_key/gtzan_key/genres"+"/"+d) if isfile(join("gtzan_key/gtzan_key/genres"+"/"+d, f))]
        for f in onlyfiles:
            os.rename("gtzan_key/gtzan_key/genres"+f"/{d}/{f}",dst_ann+f"/{f}")
    os.remove(path_gtzan_ann+"/disco.00055.pk")'''
move_data_gtzan("GTZAN",path_gtzan_audio,path_gtzan_ann)

In [5]:

path_giant_audio="Dataset/Key/GS/Audio"
path_giant_ann="Dataset/Key/GS/Ann"
path_giant_image="Dataset/Key/GS/Image_spec"
path_gtzan_audio="Dataset/Key/GTZAN/Audio"
path_gtzan_ann="Dataset/Key/GTZAN/Ann"
path_gtzan_image="Dataset/Key/GTZAN/Image_spec"
lst_name_chords=['A major','Bb major','B major','C major','Db major','D major','Eb major','E major','F major','Gb major','G major','Ab major','A minor','Bb minor','B minor','C minor','Db minor','D minor','Eb minor','E minor','F minor','Gb minor','G minor','Ab minor']
lst_chords=range(len(lst_name_chords))


def convert_to_wav(path_audio):
    onlyfiles = [f for f in listdir(path_audio) if isfile(join(path_audio, f))]
    
    #for each file, convert it into wav
    for f in onlyfiles:
        sound = AudioSegment.from_mp3(f"{path_audio}/{f}")
        dst = f"{path_audio}/{splitext(f)[0]}.wav"
        sound.export(dst, format="wav")
        os.remove(f"{path_audio}/{f}")
        
def convert_annotations(path_ann):
    #it may happen that we have annotation written like 'db minor'. We need to convert it as a number
    #It is adapted to the Dictionary of GTZAN, where tehre is only the Major and Minor scale ordered as the following list is
    
    onlyfiles = [f for f in listdir(path_ann) if isfile(join(path_ann, f))]
    for f in onlyfiles:
        a=open(f"{path_ann}/{f}","r")
        chord=a.readlines()
        a.close()
        numb_chord=lst_chords[lst_name_chords.index(chord[0])]
        a=open(f"{path_ann}/{splitext(f)[0]}.txt","w")
        a.write(str(numb_chord))
        a.close()

#giantsteps annotations file have .key extensions, let's standardize it to a classic .txt file         
def rename_key_file(path):
    #I delete the key file for faster stuff
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    for f in onlyfiles:
        os.rename(f"{path}/{f}",f"{path}/{splitext(f)[0]}.txt")
        if "lerch" in f:
            f2=((splitext(f)[0])[:len(f)-10])
            os.rename(f"{path}/{f}",f"{path}/{f2}.txt")
        
def data_augmentation(path_audio,path_ann):
    #1st data augmentation-> pitch shifting, by 2,-2,6,-6 semitones
    f_audio = [f for f in listdir(path_audio) if isfile(join(path_audio, f))]
    
    for f in f_audio:
        y, sr = librosa.load(f"{path_audio}/{f}")
        lst_name_file_shifted=["2","2n","6","6n"]
        lst_shift=[2,-2,6,-6]
        
        #memorize the chord
        a=open(f"{path_ann}/{splitext(f)[0]}.txt","r")
        chord=a.readlines()
        a.close()
        for s in lst_shift:
            #write the audio shifted
            audio=librosa.effects.pitch_shift(y, sr, n_steps=s)
            sf.write(f'{path_audio}/{lst_name_file_shifted[lst_shift.index(s)]}_{f}', audio, sr,'PCM_24')
            #modify the ann
            new_chord=lst_chords[(int(chord[0])+s) % len(lst_chords)]
            a=open(f"{path_ann}/{lst_name_file_shifted[lst_shift.index(s)]}_{splitext(f)[0]}.txt",'w')
            a.write(str(new_chord))
            a.close()
            
    #2nd data augmentation -> divide the original audio in 15 sec audio, it is needed just this lenght in order to achieve some results
    f_audio = [f for f in listdir(path_audio) if isfile(join(path_audio, f))]
    for f in f_audio:
        y,sr=librosa.load(f"{path_audio}/{f}")
        if len(y)>sr*15:
            count=1
            
            #memorize the chord
            a=open(f"{path_ann}/{splitext(f)[0]}.txt","r")
            bpm=a.readlines()
            a.close()
            
            while True:
                if(sr*15*count<=len(y)):
                    
                    y_new=y[sr*15*(count-1):sr*15*count]
                    sf.write(f'{path_audio}/{splitext(f)[0]}_{count}.wav', y_new, sr,'PCM_24')
                    if count==1: #rename the original file
                        os.rename(f"{path_ann}/{splitext(f)[0]}.txt", f"{path_ann}/{splitext(f)[0]}_{count}.txt")
                    else:
                        a=open(f"{path_ann}/{splitext(f)[0]}_{count}.txt","w")
                        a.write(str(bpm[0]))
                        a.close()
                    count+=1
                else:
                    break
                    
                    
            os.remove(f"{path_audio}/{f}")

        
rename_key_file(path_giant_ann)
rename_key_file(path_gtzan_ann)
convert_to_wav(path_giant_audio)
convert_annotations(path_giant_ann)
data_augmentation(path_giant_audio,path_giant_ann)
data_augmentation(path_gtzan_audio,path_gtzan_ann)

In [ ]:
#Preprocessing with the creation of log-melspectrogram
def creation_spectrograms(path_audio,path_img):
    
    onlyfiles = [f for f in listdir(path_audio) if isfile(join(path_audio, f))]
    for f in onlyfiles:
        y,sr=librosa.load(f"{path_audio}/{f}",sr=22050)
        ps = librosa.feature.melspectrogram(y=y,n_fft=1024, hop_length=1024//2,sr=sr,power=1,n_mels=40, fmin=20, fmax=5000)
        mels = np.log(ps + 1e-9)
        mels = librosa.power_to_db(ps, ref=np.max)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(mels, sr=sr)
        plt.savefig(f"{path_img}/{splitext(f)[0]}.jpeg")

creation_spectrograms(path_giant_audio,path_giant_image)
creation_spectrograms(path_gtzan_audio,path_gtzan_image)